#### Подключение библиотек

In [ ]:
import numpy as np
import cv2 as cv

from PIL import Image

import os
from pathlib import Path

from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import img_as_ubyte
from skimage.util import random_noise

#### Обработка видео

In [ ]:
# Auxiliary function for padding creation
def addPad(imgs, radius):
    for i in range(radius):
        imgs.insert(0, imgs[0])
        imgs.insert(-1, imgs[-1])
    return imgs


def temporalNonLocalMeans(input_name, output_name):
    # Capture video
    cap = cv.VideoCapture(input_name)

    n_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    print(f"Frames count: {n_frames}")

    w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    print(f"Video size: {w}×{h} pixels")

    fps = cap.get(cv.CAP_PROP_FPS)
    print(f"Video fps: {fps:.3} frames per second")

    # Define the codec for output video
    fourcc = cv.VideoWriter_fourcc(*"I420")

    # Set up output video
    out = cv.VideoWriter(output_name, fourcc, fps, (w, h))
    
    # Create video components lists
    imgs = []
    u = []
    v = []
    
    for i in range(n_frames):
        
        res, frame = cap.read()
        if not res:
            print(f"Frame {i+1}: End of stream")
        
        frame = cv.cvtColor(frame, cv.COLOR_BGR2YUV)
        
        imgs.append(frame[:,:, 0])
        u.append(frame[:,:, 1])
        v.append(frame[:,:, 2])
    
    # Luma component processing
    radius = 2
    imgs = addPad(imgs, radius)
    
    denoised_imgs = []
    
    for i in range(radius, n_frames + radius):
        sigma_est = np.mean(estimate_sigma(imgs[i]))
        
        denoised_imgs.append(cv.fastNlMeansDenoisingMulti(srcImgs = imgs,
                                                          imgToDenoiseIndex = i,
                                                          temporalWindowSize = 2 * radius + 1,
                                                          h = sigma_est * 1.15,
                                                          templateWindowSize = 5, 
                                                          searchWindowSize = 13))
        print(f"Frame {i - radius + 1} was denoised")
    
    print(imgs[1] - denoised_imgs[0])
    
    for i in range(len(denoised_imgs)):
        
        # Build the output denoised frame
        frame = np.zeros((h,w,3), dtype = np.uint8)
        frame[:, :, 0] = denoised_imgs[i]
        frame[:, :, 1] = u[i]
        frame[:, :, 2] = v[i]
        
        frame = cv.cvtColor(frame, cv.COLOR_YUV2BGR)
        cv.imshow("", frame)
        cv.waitKey(25)
        out.write(frame)
        print(f"Frame {i+1} was written")
    
    cap.release()
    out.release()
    cv.destroyAllWindows()

In [ ]:
str_in = "in/taolu_noise.avi"
str_out = "out/taolu_denoised_example.avi"
temporalNonLocalMeans(str_in, str_out)